In [1]:
import numpy as np
import sklearn

Question 1

    Install Pipenv
    What's the version of pipenv you installed?
    Use --version to find out

In [2]:
!pipenv --version

pipenv, version 2022.10.4



Question 2

    Use Pipenv to install Scikit-Learn version 1.0.2
    What's the first hash for scikit-learn you get in Pipfile.lock?

Models

We've prepared a dictionary vectorizer and a model.

They were trained (roughly) using this code:

features = ['reports', 'share', 'expenditure', 'owner']
dicts = df[features].to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X = dv.fit_transform(dicts)

model = LogisticRegression(solver='liblinear').fit(X, y)

    Note: You don't need to train the model. This code is just for your reference.

And then saved with Pickle. Download them:

    DictVectorizer
    LogisticRegression

With wget:

PREFIX=https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework

wget $PREFIX/model1.bin

wget $PREFIX/dv.bin

In [3]:
PREFIX = "https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework"
!wget $PREFIX"/model1.bin"
!wget $PREFIX"/dv.bin"

--2022-10-11 00:29:12--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework/model1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 889 [application/octet-stream]
Saving to: 'model1.bin'

     0K                                                       100% 37.3M=0s

2022-10-11 00:29:12 (37.3 MB/s) - 'model1.bin' saved [889/889]

--2022-10-11 00:29:12--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework/dv.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:

In [4]:
# First hash of Scikit-learn is:

print("sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b")

sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b


Question 3

Let's use these models!

    Write a script for loading these models with pickle
    Score this client:

{"reports": 0, "share": 0.001694, "expenditure": 0.12, "owner": "yes"}

What's the probability that this client will get a credit card?

- **0.148**
- 0.391
- 0.601
- 0.993

If you're getting errors when unpickling the files, check their checksum:

$ md5sum model1.bin dv.bin
3f57f3ebfdf57a9e1368dcd0f28a4a14  model1.bin
6b7cded86a52af7e81859647fa3a5c2e  dv.bin

In [5]:
import pickle

In [6]:
with open('model1.bin','rb') as f_in:
    model = pickle.load(f_in)


with open('dv.bin','rb') as f_in:
    dv = pickle.load(f_in)


In [7]:
input_data = {"reports": 0, "share": 0.001694, "expenditure": 0.12, "owner": "yes"}

In [8]:
 X = dv.transform([input_data])

In [9]:
ypred_prob = model.predict_proba(X)[0,1]
ypred_prob

0.16213414434326598

Question 4

Now let's serve this model as a web service

    Install Flask and gunicorn (or waitress, if you're on Windows)
    Write Flask code for serving the model
    Now score this client using requests:

url = "YOUR_URL"
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
requests.post(url, json=client).json()

What's the probability that this client will get a credit card?

- 0.274
- 0.484
- 0.698
- **0.928**

In [10]:
# Start the flask_app.py (shares the same folder as this file) in the command prompt/terminal,

# type the command: python flask_app.py   




# To run production-ready server, go to the same folder and 

# type the command: waitress-serve --listen 0.0.0.0:9696 flask_app:app

# and run the following code to get the model prediction

In [11]:
import requests

In [12]:
url = "http://127.0.0.1:9696/predict"
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"} 

In [13]:
response = requests.post(url, json=client).json()

In [14]:
response

{'card prediction': True, 'card probability': 0.9282218018527452}

Docker

Install Docker. We will use it for the next two questions.

For these questions, we prepared a base image: svizor/zoomcamp-model:3.9.12-slim. You'll need to use it (see Question 5 for an example).

This image is based on python:3.9.12-slim and has a logistic regression model (a different one) as well a dictionary vectorizer inside.

This is how the Dockerfile for this image looks like:

FROM python:3.9.12-slim
WORKDIR /app
COPY ["model2.bin", "dv.bin", "./"]

We already built it and then pushed it to svizor/zoomcamp-model:3.9.12-slim.

    Note: You don't need to build this docker image, it's just for your reference.

Question 5

Download the base image svizor/zoomcamp-model:3.9.12-slim. You can easily make it by using docker pull command.

So what's the size of this base image?

- 15 Mb
- **125 Mb**
- 275 Mb
- 415 Mb

You can get this information when running docker images - it'll be in the "SIZE" column.
Dockerfile

In [15]:
# Type docker images in the command prompt after installing docker and 

# running the command: docker pull svizor/zoomcamp-model:3.9.12-slim

# Run the following command to list the images: docker images

# Look for the image and size column to identify the size

**The svizor/zoomcamp-model:3.9.12-slim base image is 125MB**

Now create your own Dockerfile based on the image we prepared.

It should start like that:

FROM svizor/zoomcamp-model:3.9.12-slim
# add your stuff here

Now complete it:

    Install all the dependencies form the Pipenv file
    Copy your Flask script
    Run it with Gunicorn

After that, you can build your docker image.

Question 6

Let's run your docker container!

Aftera running it, score this client once again:

url = "YOUR_URL"
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
requests.post(url, json=client).json()

What's the probability that this client will get a credit card now?

- 0.289
- 0.502
- 0.769
- **0.972**

In [16]:
import json
import requests

In [17]:
url = "http://localhost:9696/predict"
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}

In [18]:
response = requests.post(url, json=client).json()
response

{'card prediction': True, 'card probability': 0.9282218018527452}

In [19]:
print(json.dumps(response, indent=2))

{
  "card prediction": true,
  "card probability": 0.9282218018527452
}
